**Import**

In [1]:
import random
import pandas as pd
import numpy as np
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    DonutProcessor,
    VisionEncoderDecoderConfig,
    VisionEncoderDecoderModel,
    get_scheduler
)

import wandb

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device.type

'cuda'

**Hyperparameter Settings**

In [22]:
CFG = {
    'WORKING_DIR': "/home/2023-1_DL_TeamProject_t5",
    'SEED':42,
    'NUM_WORKERS':4,
    'IMG_HEIGHT':4032,
    'IMG_WIDTH':3024,
    'MAX_LEN':1024,
    'BATCH_SIZE':2
}

**Set Working Direcotry**

In [4]:
os.chdir(CFG['WORKING_DIR'])
print(os.getcwd())

/home/2023-1_DL_TeamProject_t5


**Fix Seeds**

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

**Dataset Building & Data Preprocessing**

In [6]:
type_dict = {0:"uni", 1:"nm", 2:"ing", 3:"exp", 4:"how", 5:"des", 9:"etc"}

class DonutDataset(Dataset):

    def __init__(
        self,
        dataframe: pd.DataFrame,
        max_length: int,
        processor: DonutProcessor,
        split: str = "train",
        ignore_id: int = -100,
    ):
        super().__init__()

        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id
        self.dataframe = dataframe
        self.dataframe_length = len(self.dataframe)
        self.processor = processor
        self.gt_container = []
        
        for idx, sample in self.dataframe.iterrows():
            ground_truth = self.get_gt_strings(eval(sample['texts']))
            self.gt_container.append(ground_truth)

    def get_gt_strings(self, ct):
        
        gt_string = ""
        flag = 1
        tp = -1
        for i, item in enumerate(ct):
            if flag:
                gt_string = gt_string + f'<{type_dict[item[0]]}>'
                tp = item[0]
                flag = 0
                gt_string = gt_string + f'{item[1]}'
            
            elif not flag:
                gt_string = gt_string + f' {item[1]}'
            
            if i == len(ct)-1 or ct[i+1][0] != tp:
                gt_string = gt_string + f'</{type_dict[item[0]]}>'
                flag = 1
        
        return gt_string
    
    def __len__(self):
        
        return self.dataframe_length

    def __getitem__(self, idx: int):

        sample = self.dataframe.loc[idx]
        image = Image.open(sample['image_path'])
       
        pixel_values = self.processor(image, random_padding=self.split == "train", return_tensors="pt").pixel_values.squeeze()

        target_sequence = self.gt_container[idx] 
        input_ids = self.processor.tokenizer(
            target_sequence,
            add_special_tokens=False,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = self.ignore_id  

        return pixel_values, labels, target_sequence

In [7]:
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
processor.image_processor.size = {"height": CFG['IMG_HEIGHT'],"width": CFG['IMG_WIDTH']}

config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base")
config.encoder.image_size = [CFG['IMG_HEIGHT'], CFG['IMG_WIDTH']]
config.decoder.max_length = CFG['MAX_LEN']

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base", config=config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
added_tokens = [fr'<{x}>' for x in type_dict.values()] + [fr'</{x}>' for x in type_dict.values()]
processor.tokenizer.add_tokens(added_tokens)
model.decoder.resize_token_embeddings(len(processor.tokenizer))
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(['<s>'])[0]

In [10]:
from sklearn.model_selection import train_test_split

train_val_df = pd.read_csv("./dataframes/train_annot_df.csv")
test_df = pd.read_csv("./dataframes/test_annot_df.csv")

train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=CFG['SEED'])

train_dataset = DonutDataset(train_df, max_length=CFG['MAX_LEN'], processor=processor, split="train")
val_dataset = DonutDataset(val_df, max_length=CFG['MAX_LEN'], processor=processor, split="validation")
test_dataset = DonutDataset(test_df, max_length=CFG['MAX_LEN'], processor=processor, split="test")

In [20]:
# Sanity Check
pixel_values, labels, target_sequence = train_dataset[0]

In [16]:
print(pixel_values.shape)

torch.Size([3, 4032, 3024])


In [17]:
for id in labels.tolist()[:30]:
  if id != -100:
    print(processor.decode([id]))
  else:
    print(id)

<how>
[
사
용
시
의

주의
사항
]
1)
화
장
품
사용
시
또는
사용
후
직
사
광
선
에
의
하여
사용
부
위
가


In [18]:
print(target_sequence)

<how>[사용시의 주의사항] 1) 화장품 사용 시 또는 사용 후 직사광선에 의하여 사용부위가 붉은반점, 부어오름 또 는 가려움증 등의 이상 증상이나 부작용이 있는 경우 전문의 등과 상담할 것 2) 상처가 있는 부위 등에는 사용을 자제할 것 3) 보관 및 취급시의 주의사항 가) 어린이의 손이 닿지 않는 곳에 보관할 것 나) 직사광선을 피해서 보관할 것</how><des>품번 : 1020691</des><nm>품명 : 리더스더마소울메닛 오드퍼퓸_블루</nm><how>[사용법] 맥박이 뛰는 곳에 이를 분사하여 향기가 나게 합니다. 스프레 사용</how><des>화장품책임판매업자 : 스메틱 (주) 리더스코 경기도 안성시 미양면 제4 산단1로 34 화장품제조업자 : (주) 킨팜 경기도 김포시 월곶면 애기봉 로 456번길 72 제조번호 및</des><exp>사용기한 : 별도표기</exp><des>용량 : 30ml 소비자상담실 : 080-866-6868 · 본 제품은 공정거래위원회 고시 소비자 분쟁 해결기준에 의거 교환 받을 수 있습니다. 또는 보상 MADE IN KOREA YM DB ZM 8 808739 000306</des>


In [19]:
print("Pad token ID:", processor.decode([model.config.pad_token_id]))
print("Decoder start token ID:", processor.decode([model.config.decoder_start_token_id]))

Pad token ID: <pad>
Decoder start token ID: <s>


**Dataloader Building**

In [23]:
train_dataloader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])
val_dataloader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])
test_dataloader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])